# 이상치처리
- boxplot(IQR)
- Z-score

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [86]:
card = pd.read_csv('./data/creditcard_kaggle.csv')
card.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [87]:
card.shape

(284807, 31)

---
---

## 01. boxplot

- John Tukey가 정의한 이상치는 첫 번째 사 분위수보다 낮은 IQR의 1.5배 또는 세 번째 사 분위수보다 높은 IQR의 1.5배를 벗어나는 데이터 포인트를 "외부(outside)" 또는 "거리가 먼(far out)"것으로 간주하였다. 고전적인 박스플롯에서 검은 선으로 확장되는 '수염(whiskers)'은 박스플롯의 '외부 또는 가장자리'까지가 아닌 마지막 데이터 포인트까지 확장되어 있다
- 매우 많은 피처가 있을 경우에는 이들 중 결정갑과 가장 상관성이 높은 피처들을 위주로 이상치를 검출하는 것이 좋다.

In [92]:
# 함수정의 => data와 index 정의하지만 불러오는 건 index
def outlier_iqr(data, column): 

    # lower, upper 글로벌 변수 선언하기    
    global lower, upper    
    
    # 4분위수 기준 지정하기     
    q25, q75 = np.quantile(data[column], 0.25), np.quantile(data[column], 0.75)          
    
    # IQR 계산하기     
    iqr = q75 - q25    
    
    # outlier cutoff 계산하기     
    cut_off = iqr * 1.5          
    
    # lower와 upper bound 값 구하기     
    lower, upper = q25 - cut_off, q75 + cut_off     
    
    print('IQR은',iqr, '이다.')     
    print('lower bound 값은', lower, '이다.')     
    print('upper bound 값은', upper, '이다.')    
    
    # 1사 분위와 4사 분위에 속해있는 데이터 각각 저장하기     
    data1 = data[data[column] > upper]     
    data2 = data[data[column] < lower]
    data = pd.concat([data1,data2],axis=0)
    index = data.index
    
    # 이상치 총 개수 구하기
    print('총 이상치 개수는', data1.shape[0] + data2.shape[0], '이다.')
    return index

In [93]:
# 함수실행
outlier_iqr(card.loc[card['Class']==1],'V14')

IQR은 5.409902115485519 이다.
lower bound 값은 -17.807576138200663 이다.
upper bound 값은 3.8320323237414122 이다.
총 이상치 개수는 4 이다.


Int64Index([8296, 8615, 9035, 9252], dtype='int64')

In [94]:
# 3개의 행 줄었다
outlier_index = outlier_iqr(card.loc[card['Class']==1],'V14')
card.drop(outlier_index,axis=0,inplace=True)
card.shape

IQR은 5.409902115485519 이다.
lower bound 값은 -17.807576138200663 이다.
upper bound 값은 3.8320323237414122 이다.
총 이상치 개수는 4 이다.


(284803, 31)

---
---

## 02. Z-score
- 여기서 일반적으로 이상값을 z 점수의 계수가 임계값보다 큰 것으로 정의하는데, 이 임계값은 일반적으로 2보다 크고, 3은 공통값으로 본다. 데이터 포인트의 z 점수가 3보다 크면 데이터 포인트가 다른 데이터 포인트와 상당히 다르다는 것을 나타내게 되는데, 이러한 데이터 포인트를 이상값으로 정의한다.

- 예를 들어 한 설문 조사에서 한 사람의 형재/자매 수를 물었다고 가정하자.모든 사람으로부터 얻은 데이터가 다음과 같다고 할 때, 
    - 1, 3, 3, 2, 4, 1, 1, 12, 1, 2, 3, 2, 1, 2, 1
        전체 데이터 중에서 분명히 12는 이 데이터셋에서 이상값이다.

In [95]:
# 데이터 새로 불러오기

card = pd.read_csv('./data/creditcard_kaggle.csv')

In [96]:
# 평균과 분산 구하기

mean = np.mean(card['V14'])

std = np.std(card['V14'])

print('데이터의 평균은', mean)

print('데이터의 표준 편차는', std)

데이터의 평균은 1.479045358646473e-15
데이터의 표준 편차는 0.9585939283692662


In [104]:
# Z-score 계산하기

threshold = 3

outlier = []
index = []

for i,data in enumerate(card['V14']): 
    z = (data-mean)/std 
    if z > threshold: 
        outlier.append(data)
        index.append(i)

print('데이터셋 내의 이상값은', outlier)

데이터셋 내의 이상값은 [3.99166172232423, 3.95266350359335, 4.19701037812223, 3.1896508645056496, 3.0250998770486404, 2.9377869858226298, 2.87851871636953, 2.88023452260737, 3.10535453516329, 3.8400626696843796, 3.1541154911427998, 3.0941946749828197, 3.2848756110592205, 3.5272880950572403, 3.77296675794985, 4.123387004816791, 4.293277253516379, 4.1278319002582595, 2.94375061882015, 5.74873380036715, 3.40011632497497, 5.65025131791668, 3.19013284420896, 2.93541540772771, 3.78056480153094, 2.9878227217072797, 3.03323312623568, 3.2801747522549505, 3.5300792463709905, 3.7756834864286, 2.98093076516274, 4.01810444083434, 4.2682678190643, 3.01701479114679, 4.512944145894339, 3.3637774086816203, 4.7582074406336705, 3.18534285858362, 5.5104289853179695, 2.9998863532179, 3.2116418705263103, 2.8802946750885896, 2.96115195881118, 3.2073355278534104, 3.05851826963646, 3.0188100515888197, 3.89256716684852, 4.0480377920144806, 4.40595117839515, 3.27859780943623, 3.01644327864482, 3.1128078741540497, 3.361819

In [107]:
len(index), len(outlier)

(746, 746)

In [108]:
# 746행 줄었다
outlier_index = index
card.drop(outlier_index,axis=0,inplace=True)
card.shape

(284061, 31)

---
---